### import library

In [1]:
import pandas as pd
import numpy as np
import featuretools as ft

import warnings
warnings.filterwarnings('ignore')

### check version

In [2]:
ft.__version__

'0.16.0'

### load csv data

In [3]:
clients = pd.read_csv('./clients.csv', parse_dates = ['joined'])
clients.head()

,client_id,joined,income,credit_score
0,46109,2002-04-16,172677,527
1,49545,2007-11-14,104564,770
2,41480,2013-03-11,122607,585
3,46180,2001-11-06,43851,562
4,25707,2006-10-06,211422,621


In [4]:
clients.isna().sum()

client_id       0
joined          0
income          0
credit_score    0
dtype: int64

In [5]:
loans = pd.read_csv('./loans.csv', parse_dates = ['loan_start', 'loan_end'])
loans.head()

,client_id,loan_type,loan_amount,repaid,loan_id,loan_start,loan_end,rate
0,46109,home,13672,0,10243,2002-04-16,2003-12-20,2.15
1,46109,credit,9794,0,10984,2003-10-21,2005-07-17,1.25
2,46109,home,12734,1,10990,2006-02-01,2007-07-05,0.68
3,46109,cash,12518,1,10596,2010-12-08,2013-05-05,1.24
4,46109,credit,14049,1,11415,2010-07-07,2012-05-21,3.13


In [6]:
loans.isna().sum()

client_id      0
loan_type      0
loan_amount    0
repaid         0
loan_id        0
loan_start     0
loan_end       0
rate           0
dtype: int64

In [7]:
payments = pd.read_csv('./payments.csv', parse_dates = ['payment_date'])
payments.head()

,loan_id,payment_amount,payment_date,missed
0,10243,2369,2002-05-31,1
1,10243,2439,2002-06-18,1
2,10243,2662,2002-06-29,0
3,10243,2268,2002-07-20,0
4,10243,2027,2002-07-31,1


In [8]:
payments.isna().sum()

loan_id           0
payment_amount    0
payment_date      0
missed            0
dtype: int64

### Define entity

In [9]:
es = ft.EntitySet(id='clients')

es = es.entity_from_dataframe(entity_id='clients',
                                dataframe=clients,
                                index = 'client_id',
                                time_index='joined')

In [10]:
es = es.entity_from_dataframe(entity_id='loans',
                                dataframe=loans,
                                index = 'loan_id',
                                time_index='loan_start')

In [11]:
es = es.entity_from_dataframe(entity_id='payments',
                              dataframe=payments,
                              variable_types={'missed':
                                              ft.variable_types.Categorical},
                                              make_index=True,
                                              index='payment_id',
                                              time_index='payment_date')

### Check entity

In [12]:
es

Entityset: clients
  Entities:
    clients [Rows: 25, Columns: 4]
    loans [Rows: 443, Columns: 8]
    payments [Rows: 3456, Columns: 5]
  Relationships:
    No relationships

In [13]:
es['loans']

Entity: loans
  Variables:
    loan_id (dtype: index)
    client_id (dtype: numeric)
    loan_type (dtype: categorical)
    loan_amount (dtype: numeric)
    repaid (dtype: numeric)
    loan_start (dtype: datetime_time_index)
    loan_end (dtype: datetime)
    rate (dtype: numeric)
  Shape:
    (Rows: 443, Columns: 8)

### create just one new feature (total_loan_amount)

In [14]:
stats = loans.groupby('client_id')['loan_amount'].agg(['sum'])
stats.columns = ['total_loan_amount']

stats = clients.merge(stats, left_on='client_id',right_index=True,how='left')

stats.head(10)

,client_id,joined,income,credit_score,total_loan_amount
0,46109,2002-04-16,172677,527,179032
1,49545,2007-11-14,104564,770,205786
2,41480,2013-03-11,122607,585,157897
3,46180,2001-11-06,43851,562,154017
4,25707,2006-10-06,211422,621,159279
5,39505,2011-10-14,153873,610,148481
6,32726,2006-05-01,235705,730,126032
7,35089,2010-03-01,131176,771,138784
8,35214,2003-08-08,95849,696,129124
9,48177,2008-06-09,190632,769,141063


### define Relationship

In [15]:
r_client_previous = ft.Relationship(es['clients']['client_id'],es['loans']['client_id'])

es = es.add_relationship(r_client_previous)

r_payments = ft.Relationship(es['loans']['loan_id'],es['payments']['loan_id'])

es = es.add_relationship(r_payments)
es

Entityset: clients
  Entities:
    clients [Rows: 25, Columns: 4]
    loans [Rows: 443, Columns: 8]
    payments [Rows: 3456, Columns: 5]
  Relationships:
    loans.client_id -> clients.client_id
    payments.loan_id -> loans.loan_id

### do dfs

In [16]:
features, feature_names = ft.dfs(entityset= es,
                                 target_entity='clients',
                                 agg_primitives=['mean', 'max','percent_true','last'],
                                 trans_primitives=['year', 'month', 'subtract_numeric', 'divide_numeric'])


### check data

In [17]:
pd.DataFrame(features['MONTH(joined)'].head())

,MONTH(joined)
client_id,
42320,4
39384,6
26945,11
41472,11
46180,11


In [18]:
pd.DataFrame(features['MEAN(payments.payment_amount)'].head())

,MEAN(payments.payment_amount)
client_id,
42320,1021.483333
39384,1193.630137
26945,1109.473214
41472,1129.076190
46180,1186.550336


In [19]:
pd.DataFrame(features[['MEAN(loans.loan_amount)','MEAN(loans.rate)','MAX(loans.loan_amount)','MAX(loans.rate)','LAST(loans.loan_type)','LAST(loans.loan_amount)']].head())

,MEAN(loans.loan_amount),MEAN(loans.rate),MAX(loans.loan_amount),MAX(loans.rate),LAST(loans.loan_type),LAST(loans.loan_amount)
client_id,,,,,,
42320,7062.066667,2.457333,13887,6.74,home,8090
39384,7865.473684,3.538421,14654,9.23,other,14654
26945,7125.933333,2.855333,14593,5.65,cash,9249
41472,7510.812500,3.981250,13657,9.82,cash,10122
46180,7700.850000,3.502500,14081,9.26,other,3834
